In [1]:
import glob
import os

import anndata as ad
import scanpy as sc
from scipy.sparse import csr_matrix


from src.da_utils.data_processing import qc_sc


In [2]:
SPOTLESS_DIR = "data/spotless/standards"

CORTEX_DIR = "data/mouse_cortex"
OLFACTORY_DIR = "data/mouse_olfactory"
# SC_ID_OLFACTORY = "spotless_mouse_olfactory"
# SC_ID_CORTEX = "spotless_mouse_cortex"
# SC_ID_VISUAL = "spotless_mouse_visual"


# %%

standard_to_id = {
    "gold_standard_1": "spotless_mouse_cortex",
    "gold_standard_2": "spotless_mouse_olfactory",
    "gold_standard_3": "spotless_mouse_visual",
    "gold_standard_3_12celltypes": "spotless_mouse_visual",
}

id_to_dir = {
    "spotless_mouse_cortex": CORTEX_DIR,
    "spotless_mouse_olfactory": OLFACTORY_DIR,
    "spotless_mouse_visual": CORTEX_DIR,
}



In [45]:
def cat_to_obsm(cat, adata, drop_cols=None):
    if drop_cols is None:
        drop_cols = []
    selected_cols = adata.obs.columns.to_series().map(lambda x: x.split(".")[0] == cat)
    adata.obsm[cat] = (
        adata.obs.loc[:, selected_cols]
        .rename(columns=lambda x: x[len(cat + ".") :])
        .drop(columns=drop_cols)
    )
    keep_cols = ~selected_cols
    # print(keep_cols)

    for drop_col in drop_cols:
        keep_cols[cat + "." + drop_col] = True
    adata.obs = adata.obs.loc[:, keep_cols]

fpaths = sorted(glob.glob(os.path.join(SPOTLESS_DIR, "gold_standard_1", "*.h5ad")))
sample_ids = [os.path.basename(f).split(".")[0] for f in fpaths]
fovs = [sc.read_h5ad(name) for name in fpaths]

obs_cols = sorted(list(set.union(*[set(fov.obs.columns) for fov in fovs])))
for fov, sample_id in zip(fovs, sample_ids):
    fov.obs = fov.obs.reindex(columns=obs_cols)
    fov.obs = fov.obs.fillna(0)
    fov.obs = fov.obs.rename(columns={"coordinates.x": "X", "coordinates.y": "Y"})
    fov.X = csr_matrix(fov.X.astype("float32"))
    fov.raw = fov

    fov.obs = fov.obs.loc[:,~fov.obs.columns.str.contains("spot_no")]

    cat_to_obsm("relative_spot_composition", fov)
    cat_to_obsm("spot_composition", fov)

In [49]:
fovs[0].obsm["relative_spot_composition"]

,Astrocytes.deep,Astrocytes.superficial,Choroid.plexus,Endothelial,Ependymal,Excitatory.layer.3,Excitatory.layer.4,Excitatory.layer.5.6,Excitatory.layer.II,Interneurons,Interneurons.deep,Microglia,NSC,Neural.progenitors,Neuroblasts,OPC,Oligodendrocytes
spot_1,0.000000,0.000000,0.0,0.058824,0.0,0.176471,0.235294,0.0,0.352941,0.117647,0.000000,0.058824,0.0,0.0,0.0,0.000000,0.0
spot_2,0.076923,0.076923,0.0,0.000000,0.0,0.076923,0.153846,0.0,0.461538,0.076923,0.000000,0.076923,0.0,0.0,0.0,0.000000,0.0
spot_3,0.000000,0.333333,0.0,0.166667,0.0,0.000000,0.000000,0.0,0.166667,0.166667,0.166667,0.000000,0.0,0.0,0.0,0.000000,0.0
spot_4,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.142857,0.0,0.571429,0.285714,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
spot_5,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.636364,0.181818,0.000000,0.000000,0.0,0.0,0.0,0.181818,0.0
spot_6,0.000000,0.800000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.200000,0.0,0.0,0.0,0.000000,0.0
spot_7,0.000000,0.000000,0.0,0.000000,0.0,0.222222,0.222222,0.0,0.333333,0.111111,0.000000,0.111111,0.0,0.0,0.0,0.000000,0.0
spot_8,0.000000,0.000000,0.0,0.083333,0.0,0.000000,0.083333,0.0,0.583333,0.250000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
spot_9,0.000000,0.000000,0.0,0.500000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.500000,0.0,0.0,0.0,0.000000,0.0


In [55]:
for ref_path in glob.glob(os.path.join(SPOTLESS_DIR, "reference", "gold_standard_1.h5ad")):
    if "19celltypes" in ref_path:
        continue

    name = os.path.basename(ref_path).split(".")[0]
    id = standard_to_id[name]

    dset_dir = id_to_dir[id]
    print(f"Processing {name} to {id} in {dset_dir}")

    sc_dir = os.path.join(dset_dir, "sc_adata")
    if not os.path.exists(sc_dir):
        os.makedirs(sc_dir)

    adata_sc = sc.read_h5ad(ref_path)

    qc_sc(adata_sc)

    adata_sc.obs = adata_sc.obs.rename(columns={"celltype": "cell_subclass"})

    adata_sc.X = csr_matrix(adata_sc.X.astype("float32"))
    adata_sc.raw = adata_sc

    break

    # adata_sc.write(os.path.join(sc_dir, f"{id}.h5ad"))


Processing gold_standard_1 to spotless_mouse_cortex in data/mouse_cortex
0 mitochondrial genes


In [59]:
import re



{'Excitatory.layer.3': 'Excitatory layer 3',
 'Excitatory.layer.II': 'Excitatory layer II',
 'Interneurons': 'Interneurons',
 'Excitatory.layer.4': 'Excitatory layer 4',
 'Microglia': 'Microglia',
 'Astrocytes.deep': 'Astrocytes deep',
 'Endothelial': 'Endothelial',
 'Interneurons.deep': 'Interneurons deep',
 'Astrocytes.superficial': 'Astrocytes superficial',
 'OPC': 'OPC',
 'Excitatory.layer.5.6': 'Excitatory layer 5/6',
 'Neuroblasts': 'Neuroblasts',
 'NSC': 'NSC',
 'Oligodendrocytes': 'Oligodendrocytes',
 'Ependymal': 'Ependymal',
 'Neural.progenitors': 'Neural progenitors',
 'Choroid.plexus': 'Choroid plexus'}

In [69]:
adata = sc.read_h5ad("data/mouse_cortex/sc_adata/spotless_mouse_cortex.h5ad")

In [70]:
adata.X.toarray().max()

150.0

In [71]:
adata.raw.X.toarray().max()

150.0

In [5]:
for fov in fovs:
    fov.obs = fov.obs.reindex(columns=obs_cols)

In [6]:
# for column in fovs[0].obs.columns:
#     display(fovs[0].obs[column])
[os.path.basename(f).split(".")[0] for f in fpaths]

['Eng2019_cortex_svz_fov0',
 'Eng2019_cortex_svz_fov1',
 'Eng2019_cortex_svz_fov2',
 'Eng2019_cortex_svz_fov3',
 'Eng2019_cortex_svz_fov4',
 'Eng2019_cortex_svz_fov5',
 'Eng2019_cortex_svz_fov6']

In [12]:
gs1_ref_path = "data/spotless/reference/gold_standard_1.h5ad"

In [13]:
gs1_ref = sc.read_h5ad(gs1_ref_path)

In [14]:
gs1_ref

AnnData object with n_obs × n_vars = 906 × 10000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'louvain', 'celltype', 'Field.of.View', 'X', 'Y', 'celltype_coarse', 'Region'
    var: 'features'

In [6]:
# adata.obs

In [11]:
# result = pyreadr.read_r('data/spotless/gold_standard_1/Eng2019_cortex_svz_fov0.rds')
with h5py.File("data/spotless/gold_standard_1.h5" , "r") as f:
    for name in f:
        gene_names = [str(gene, "utf-8") for gene in f[name]["geneNames"][()]]
        ad.AnnData(csr_matrix(f[name]["counts"][()]), dtype=np.int64)

AnnData object with n_obs × n_vars = 9 × 10000
AnnData object with n_obs × n_vars = 9 × 10000
AnnData object with n_obs × n_vars = 9 × 10000
AnnData object with n_obs × n_vars = 9 × 10000
AnnData object with n_obs × n_vars = 9 × 10000
AnnData object with n_obs × n_vars = 9 × 10000
AnnData object with n_obs × n_vars = 9 × 10000
